In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('/content/Sentiment.csv')
data.head()

id               candidate  ...  tweet_location               user_timezone
0   1  No candidate mentioned  ...             NaN                       Quito
1   2            Scott Walker  ...             NaN                         NaN
2   3  No candidate mentioned  ...             NaN                         NaN
3   4  No candidate mentioned  ...           Texas  Central Time (US & Canada)
4   5            Donald Trump  ...             NaN                     Arizona

[5 rows x 21 columns]

In [ ]:
print(data.groupby('sentiment').nunique())

             id  candidate  ...  tweet_location  user_timezone
sentiment                   ...                               
Negative   8493         11  ...            2926             87
Neutral    3142         11  ...            1419             70
Positive   2236         11  ...            1020             58

[3 rows x 20 columns]


In [ ]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data.head()

text sentiment
0  RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1  RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2  RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3  RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4  RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive

In [ ]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])

RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate
RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.


In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower()) # all string to lowercase Read more about lambda() "https://realpython.com/python-lambda/"
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x))) # using regular expression preprocess the text by removing everything that is not [a-zA-z0-9\s]

In [ ]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])

rt nancyleegrahn how did everyone feel about the climate change question last night exactly gopdebate
rt scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff
rt tjmshow no mention of tamir rice and the gopdebate was held in cleveland wow


In [ ]:
for idx, row in data.iterrows(): # Iterate over DataFrame rows as (index, Series) pairs.
    row[0] = row[0].replace('rt', '') # replace/ remove 'rt' in the start of the text

In [ ]:
print(data['text'][0])
print(data['text'][1])
print(data['text'][2])

 nancyleegrahn how did everyone feel about the climate change question last night exactly gopdebate
 scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff
 tjmshow no mention of tamir rice and the gopdebate was held in cleveland wow


In [ ]:
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)

In [ ]:
# Transforms each text in texts to a sequence of integers.
# Only top num_words-1 most frequent words will be taken into account. Only words known by the tokenizer will be taken into account.

# Transforms each text in texts to a sequence of integers. 
# So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.
X = tokenizer.texts_to_sequences(data['text'].values)

In [ ]:
# tokenizer.get_config()
# Returns the tokenizer configuration as Python dictionary.
print(tokenizer.get_config().keys())

# word_counts: A dictionary of words and their counts.
print("\ntokenizer.word_counts") 
print(tokenizer.word_counts) 

# document_count:An integer count of the total number of documents that were used to fit the Tokenizer.
print("\ntokenizer.document_count")
print(tokenizer.document_count) 

# word_index: A dictionary of words and their uniquely assigned integers.
print("\ntokenizer.word_index")
print(tokenizer.word_index)

# word_docs: A dictionary of words and how many documents each appeared in.
print("\ntokenizer.word_docs")
print(tokenizer.word_docs)



print('\nLen() of X:', len(X))
print('\n', X[:2])

dict_keys(['num_words', 'filters', 'lower', 'split', 'char_level', 'oov_token', 'document_count', 'word_counts', 'word_docs', 'index_docs', 'index_word', 'word_index'])

tokenizer.word_counts
OrderedDict([('nancyleegrahn', 6), ('how', 623), ('did', 434), ('everyone', 91), ('feel', 64), ('about', 1146), ('the', 8903), ('climate', 72), ('change', 84), ('question', 350), ('last', 888), ('night', 632), ('exactly', 22), ('gopdebate', 9014), ('scottwalker', 88), ('didnt', 270), ('catch', 9), ('full', 45), ('here', 145), ('are', 820), ('some', 167), ('of', 3376), ('scotts', 2), ('best', 185), ('lines', 14), ('in', 1986), ('90', 12), ('seconds', 15), ('walker16', 28), ('httptcozsff', 1), ('tjmshow', 1), ('no', 528), ('mention', 56), ('tamir', 2), ('rice', 2), ('and', 2896), ('was', 1149), ('held', 10), ('cleveland', 37), ('wow', 53), ('robgeorge', 1), ('that', 1445), ('carly', 118), ('fiorina', 140), ('is', 3356), ('trending', 2), ('hours', 34), ('after', 348), ('her', 192), ('debate', 1051), 

In [ ]:
# Check the sequence of the text, Do we need to Pad ??
for i in range(4):
  print(X[i])
  print('len=', len(X[i]))

[52, 78, 338, 449, 22, 2, 413, 361, 95, 29, 51, 1023, 1]
len= 13
[348, 123, 1937, 2, 588, 1, 29, 51, 226, 35, 195, 5, 172, 1400, 10, 1561, 1336, 833]
len= 18
[62, 491, 5, 8, 2, 1, 21, 1781, 10, 685, 512]
len= 11
[17, 276, 235, 6, 723, 96, 164, 24, 130, 5, 2, 176, 10, 1, 211, 773, 16]
len= 17


In [ ]:
X = pad_sequences(X) # Pads sequences to the same length.
print('X.shape = ', X.shape)

X.shape =  (13871, 28)


In [ ]:
# Check the sequence after padding, Which padding pre or post??
for i in range(4):
  print(X[i])
  print('len=', len(X[i]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0   52   78  338  449   22    2  413  361   95   29   51 1023    1]
len= 28
[   0    0    0    0    0    0    0    0    0    0  348  123 1937    2
  588    1   29   51  226   35  195    5  172 1400   10 1561 1336  833]
len= 28
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   62  491    5    8    2    1   21 1781   10  685  512]
len= 28
[  0   0   0   0   0   0   0   0   0   0   0  17 276 235   6 723  96 164
  24 130   5   2 176  10   1 211 773  16]
len= 28


In [ ]:
embed_dim = 128
lstm_out = 196

def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.summary()
    return model

In [ ]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [ ]:
model = createmodel()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 128)           256000    
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 1)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

Epoch 1/7
291/291 [==============================] - 76s 170ms/step - loss: 0.8974 - accuracy: 0.6101
Epoch 2/7
291/291 [==============================] - 50s 173ms/step - loss: 0.6577 - accuracy: 0.7238
Epoch 3/7
291/291 [==============================] - 50s 172ms/step - loss: 0.5987 - accuracy: 0.7468
Epoch 4/7
291/291 [==============================] - 50s 171ms/step - loss: 0.5429 - accuracy: 0.7794
Epoch 5/7
291/291 [==============================] - 49s 170ms/step - loss: 0.4977 - accuracy: 0.8022
Epoch 6/7
291/291 [==============================] - 50s 172ms/step - loss: 0.4656 - accuracy: 0.8084
Epoch 7/7
291/291 [==============================] - 50s 171ms/step - loss: 0.4159 - accuracy: 0.8362
144/144 - 3s - loss: 0.9224 - accuracy: 0.6555
0.9224476218223572
0.6555264592170715
['loss', 'accuracy']
